# Sampling the mean performance when using less seeds

How close do we get to the expected performance of 100 seeds when we sample 5, 10, 15, 20, ... random seeds from our experiments? How many different seeds should we run to verify our experiments?

This notebook is unfinished

In [1]:
import matplotlib as plt
import numpy as np

from scipy import stats

from src.metrics import *

In [2]:
datapath = "/datadrive/experiments"
metrics = ["MAPE"]

In [3]:
da_elec = da_elec = load_data(f"{datapath}/deep_ar/electricity/5A")
da_elec_arr = calculate_timeseries_means(da_elec, metrics, 321)
da_elec_means = da_elec_arr.mean(axis=1)

In [4]:
def ratio_bounds(means, lower_bound, upper_bound, num_repetitions_per_sample, num_samples=100000):
    samples = np.random.randint(0, 100, size=(num_samples, num_repetitions_per_sample))
    trues = 0
    for idx in samples:
        mean = means[idx].mean()
        if lower_bound < mean < upper_bound:
            trues += 1
    return trues / num_samples

In [5]:
lower_bound = da_elec_means.mean() - da_elec_means.std()
upper_bound = da_elec_means.mean() + da_elec_means.std()

lower_bound, upper_bound

(0.09182808441008568, 0.1437868060675496)

In [6]:
for num_reps_per_sample in [5, 10, 20, 40, 60, 80, 100]:
    print(f"{num_reps_per_sample}: {ratio_bounds(da_elec_means, lower_bound, upper_bound, num_reps_per_sample)}")

5: 0.95819
10: 0.99162
20: 0.99945
40: 1.0
60: 1.0
80: 1.0
100: 1.0


In [14]:
# Is the mean of the n samples identical to the true mean?
def ratio_ttest_1samp(means, true_mean, num_repetitions_per_sample, num_samples=100000):
    samples = np.random.randint(0, 100, size=(num_samples, num_repetitions_per_sample))
    trues = 0
    for idx in samples:
        p_value = stats.ttest_1samp(da_elec_means[idx], true_mean)[-1][0]
        if p_value > 0.05:
            trues += 1
    return trues / num_samples

In [15]:
### Do the individual repetitions get a high p-value when comparing it to the mean for all repetitions?
stats.ttest_1samp(da_elec_means, da_elec_means.mean())

Ttest_1sampResult(statistic=array([0.]), pvalue=array([1.]))

In [16]:
for num_reps_per_sample in [5, 10, 20, 40, 60, 80, 100]:
    print(f"{num_reps_per_sample}: {ratio_ttest_1samp(da_elec_means, da_elec_means.mean(), num_reps_per_sample)}")

5: 0.81305
10: 0.84473
20: 0.87886
40: 0.89312
60: 0.90753
80: 0.91899
100: 0.92528
